# Plate Cutting - An Optimization Approach

The purpose of this notebook is to analyze data from a plate cutter in a manufacturing environment and find ways to improve efficiency. It documents the full process of loading and cleaning raw CSV files to analyzing each column. We'll process and visualize the data to identify trends and inefficiencies. The goal is to turn raw machine logs into meaningful insights to help to optimize the cutting process.

## Pre-processing

We will check that all csv's have the same column names, and then we will concatenate all of the csv's into a single file.

In [13]:
# Files are located in data/landing folder. We must ensure all files have the same column names or we wont be able to merge them.

import pandas as pd
import os

# Variables
landing_folder = "../data/landing/files"
columns = ['ProgramName', 'Start', 'End', 'Cancelled', 'TotalTime', 'BreakingTime', 'NumberBreaks', 'Process1Time', 'Process1Length', 'Process1Starts', 'Process2Time', 'Process2Length', 'Process2Starts', 'Process3Time', 'Process3Length', 'Process3Starts', 'Process4Time', 'Process4Length', 'Process4Starts', 'RapidTime', 'RapidLength', 'RapidStarts']


# Function to load a file into a pandas dataframe
def load_file(*args, **kwargs):
    try:
        return pd.read_csv(*args, **kwargs)
    except Exception as e:
        print(f"Error loading file: {args[0]}")
        print(e)


# Function to read a file and compare name of columns with list of columns we expect
def check_column_names(file, columns):
    df = load_file(file, delimiter=';')

    if not all([col in df.columns for col in columns]):
        print(f"Columns in {file} do not match expected columns")
        print(f"Missing columns: {set(columns) - set(df.columns)}")
        print(f"Extra columns: {set(df.columns) - set(columns)}\n\n")


# check if all files have correct columns
for file in os.listdir(landing_folder):
    check_column_names(f"{landing_folder}/{file}", columns)

In [ ]:
# Variables
concatenated_file_path = "../data/landing/concat_file/full_file.csv"


# Updated merge function using pd.concat()
def merge_files(landing_folder):
    all_files = [os.path.join(landing_folder, file) for file in os.listdir(landing_folder) if file.endswith(".csv")]
    
    df_list = [load_file(file, delimiter=";") for file in all_files]
    df = pd.concat(df_list, ignore_index=True)
    
    return df


def create_folder(folder):
    os.makedirs(folder, exist_ok=True)


def save_file(df, file):
    create_folder(os.path.dirname(file))
    df.to_csv(file, index=False, sep=";")


df = merge_files(landing_folder)
save_file(df, concatenated_file_path)
print(f"File saved: {concatenated_file_path}")


File saved: ../data/landing/concat_file/full_file.csv


In [ ]:
# Variables
desired_columns = [ #will use this list after cleaning the data to ensure correct processing
    "ProgramName", "Nest ID", "Start", "Nest start time", "End", "Nest end time", "Cancelled", "Nest cancelled Yes/No",
    "TotalTime", "BreakingTime", "NumberBreaks", "Process1Time", "Process1Length", "Process1Starts", "RapidTime",
    "RapidLength", "RapidStart"
]
cleaned_file_path = "../data/staging/cleaned_file.csv"

# True if any value is zero, drops columns with all zeros
df = df.loc[:, (df != 0).any(axis=0)]

# Check if 'ProgramName' contains only numeric values
df = df[df['ProgramName'].astype(str).str.isnumeric()] # returns true if all characters in the string are numeric
df['ProgramName'] = df['ProgramName'].astype(int)

# Remove duplicate ProgramName rows, keep the first occurrence
df = df.drop_duplicates(subset=['ProgramName'], keep='first')

df = df[df['Cancelled'] != True] # remove rows where 'Cancelled' is True

# Remove rows where key columns contain zero
df = df[(df[['BreakingTime', 'Process1Time', 'Process1Length', 'RapidLength']] != 0).all(axis=1)]

# Check if remaining columns match the desired list
remaining_columns = df.columns

if remaining_columns != desired_columns:
    missing_columns = desired_columns - remaining_columns
    extra_columns = remaining_columns - desired_columns
    print(f"Warning: Column mismatch detected!\nMissing: {missing_columns}\nExtra: {extra_columns}")
else:
    print("Column validation successful. All required columns are present.")

# Save the cleaned data
save_file(df, cleaned_file_path)

print(f"Cleaned file saved: {cleaned_file_path}")